# Searching & downloading *Kepler*, *K2*, and *TESS* data

## Learning Goals

By the end of this tutorial, you will:

* Understand the data products available to query and download.
* Be able to use [Lightkurve](https://docs.lightkurve.org) to search for *Kepler*/*K2* and *TESS* data products.
* Know how to download *TESS* Full Frame Image cutouts.
* Be able to perform a cone search.

## Introduction

The [Lightkurve](https://docs.lightkurve.org) Python package has functions to search for and download observations from *Kepler*/*K2* and *TESS*. These tools are built to make accessing space telescope data clear and straightforward, with intuitive method and keyword names.

This tutorial outlines what data products are available to query with Lightkurve, and gives examples of how to use the functions to search for and download space telescope observations.

## Imports

This tutorial requires the [**Lightkurve**](http://docs.lightkurve.org/) package, which uses [**Matplotlib**](https://matplotlib.org/) for plotting.

In [ ]:
import lightkurve as lk
import matplotlib.pyplot as plt
%matplotlib inline

---

## 1. What Data Products are Available?

*Kepler*/*K2* and *TESS* data products are stored on the Mikulski Archive for Space Telescopes (MAST) in two main forms:

* **Light curve products**: tables containing the measured flux at each observation time.
* **Target pixel file products**: stacks of images with the pixel-level observation at each observation time.

There are also the following additional products available to query and download using [Lightkurve](http://docs.lightkurve.org/):

* **High Level Science Products (HLSPs)**: a specific version of a data product produced by an analysis or photometry pipeline. Lightkurve has access to HLSP light curves produced by the photometry pipelines [EVEREST](https://archive.stsci.edu/hlsp/everest), [K2SFF](https://archive.stsci.edu/hlsp/k2sff), and [K2SC](https://arxiv.org/abs/1603.09167). For more information about HLSPs, please see [this article](https://archive.stsci.edu/hlsp/) on the Space Telescope Science Institute's archive.
* **Full Frame Images (FFIs)**: a download of all active detector pixels at once. *TESS* FFIs are captured with 30-minute cadence, and custom cutouts of *TESS* FFIs can be queried and downloaded using [Lightkurve](http://docs.lightkurve.org/).

Lightkurve allows you to query and download each of these data products. The following sections contain examples of how to use the search functions in Lightkurve.

## 2. Searching for Light Curves

Lightkurve uses [Astroquery](https://astroquery.readthedocs.io/en/latest/) to search for data products. Astroquery allows searches based on a target's coordinates, catalog ID number, or name.

This is passed into the search function using the `target` keyword, and all valid inputs for identifying a target include:
    
  * The name of the object as a string, for example, "Kepler-10."
  * The KIC or EPIC identifier as an integer, for example, "11904151."
  * A coordinate string in decimal format, for example, "285.67942179 +50.24130576."
  * A coordinate string in sexagesimal format, for example, "19:02:43.1 +50:14:28.7."
  * An `astropy.coordinates.SkyCoord` object.

You can also specify which mission you would like to retrieve data from using the `mission` keyword, which takes "Kepler," "K2," or "TESS." By default, all available missions will be returned.

We will start with the case of searching for a *Kepler* target using its Kepler Input Catalog (KIC) ID number. Below, we search for [KIC 3733346](https://arxiv.org/pdf/1001.0417.pdf), an [RR Lyrae star](https://en.wikipedia.org/wiki/RR_Lyrae_variable), using the [search_lightcurve](https://docs.lightkurve.org/reference/api/lightkurve.search_lightcurve.html?highlight=search_lightcurve) function.

In [ ]:
search_result = lk.search_lightcurve('KIC 3733346', author='Kepler')
search_result

`search_lightcurve` returns a `SearchResult` table, which contains information about the data products available to download. This search result tells us that KIC 3733346 was observed in *Kepler* Quarters 1–17. 

You can select an individual entry in this search result by indexing the search result.

In [ ]:
search_result[0]

For more information about the available data products, the `SearchResult` has a full table accessible by calling [.table](https://docs.lightkurve.org/reference/api/lightkurve.SearchResult.table.html?highlight=table). This full table contains the columns listed below. Definitions of each of these terms can be found [here](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html).

In [ ]:
for column in search_result.table.columns:
  print(column)

These column names can also be used to search for specific entries in the table.

In [ ]:
# import numpy, which we will use to find the desired index in the table
import numpy as np
quarter2_index = np.where(search_result.table['mission'] == 'Kepler Quarter 02')[0]
search_result[quarter2_index]

You can also narrow down the list of observations when you make the search using the following mission-specific keywords: 
* *Kepler*: `quarter` 
* *K2*: `campaign`
* *TESS*: `sector`

Let's perform the search for KIC 3733346 again, this time specifying that we only want data from *Kepler* Quarter 2.

In [ ]:
search_result_q2 = lk.search_lightcurve('KIC 3733346', author='Kepler', quarter=2)
search_result_q2

### 2.1 Downloading a single light curve

A light curve can be downloaded by calling [.download()](https://docs.lightkurve.org/reference/api/lightkurve.SearchResult.download.html?highlight=download#lightkurve.SearchResult.download). 

In [ ]:
lc = search_result_q2.download()
lc

This returns a single `KeplerLightCurve` object, which is shown above in the form of an `astropy` table. We can examine the light curve using the `plot` method.

In [ ]:
lc.plot();

### 2.2 Downloading a collection of light curves

The `SearchResult` object also has a [download_all](https://docs.lightkurve.org/reference/api/lightkurve.SearchResult.download_all.html?highlight=download_all) method, allowing you to download multiple light curves. This returns a `LightCurveCollection`, a convenient container for `LightCurve` objects.

In [ ]:
lc_collection = search_result[:5].download_all()
lc_collection

The `LightCurveCollection` has a number of useful functions for plotting and manipulating the light curves. For more information about how to combine multiple light curves, please see the tutorial on combining multiple quarters of *Kepler* observations.

One of the methods the collection enables you to use is `plot`, making it possible to quickly visualize all observations in your collection.

In [ ]:
# Create a larger figure for clarity
fig, ax = plt.subplots(figsize=(20,5))
# Plot the light curve collection
lc_collection.plot(ax=ax);

You can also iterate through a collection to label them more clearly and to perform additional actions like normalization.

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
for lc in lc_collection:
  lc.normalize().plot(ax=ax, label=f'Quarter {lc.quarter}');

## 3. Searching for Target Pixel Files

The other primary data product used by Lightkurve is the [TargetPixelFile](https://heasarc.gsfc.nasa.gov/docs/tess/data-products.html#target-pixel-files-tpfs), or TPF. A TPF is a stack of images containing the flux in each pixel at each cadence.

Similar to the approach above, we can use the [search_targetpixelfile](https://docs.lightkurve.org/reference/api/lightkurve.search_targetpixelfile.html?highlight=search_targetpixelfile) method to identify available observations.

In [ ]:
search_result = lk.search_targetpixelfile('K2-199', exptime=1800)
search_result

This returns a table which contains the same information as a light curve search result.

### 3.1 Downloading a single target pixel file

When you call `download` on a search result containing more than one entry, it will download only the first entry in the search result. Lightkurve will raise a friendly warning to let you know when this occurs.

In [ ]:
tpf = search_result.download()

We can view a single cadence of the TPF using the `plot` method.

In [ ]:
tpf.plot();

If we want to turn the TPF into a light curve, there is a [to_lightcurve](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.to_lightcurve.html?highlight=to_lightcurve) method. 

In [ ]:
lc = tpf.to_lightcurve()
lc.plot();

For more information about using and plotting TPFs, please see the tutorials on using *Kepler* target pixel file products with Lightkurve and plotting *Kepler* target pixel file products with Lightkurve.

### 3.2 Downloading a collection of target pixel files

You can also download multiple TPFs at a time using the `download_all` method, which returns a `TargetPixelFileCollection`.

In [ ]:
tpf_collection = search_result.download_all()
tpf_collection

A single cadence of each of these TPFs can be inspected with the `plot` method.

In [ ]:
tpf_collection.plot();

## 4. Searching for *TESS* Full Frame Image (FFI) Cutouts

It is also possible to download targets observed in the *TESS* Full Frame Images (FFIs) using Lightkurve. This is done using [search_tesscut](https://docs.lightkurve.org/reference/api/lightkurve.search_tesscut.html?highlight=search_tesscut), which utilizes the [TESSCut](https://mast.stsci.edu/tesscut/) tool ([Brasseur et. al 2019](https://ui.adsabs.harvard.edu/abs/2019ascl.soft05007B/abstract)).

In [ ]:
search_result = lk.search_tesscut('Pi Men')
search_result

*TESS* FFI cutouts are downloaded as `TargetPixelFile` objects. This is done using the same download function as above, but it now takes an additional argument `cutout_size`, which describes the number of pixels along the side of the cutout, and can be an `int` or a `tuple`.

In [ ]:
tpf_cutout = search_result[0].download(cutout_size=10)
tpf_cutout.plot();

## 5. Performing a Cone Search

If you are interested in identifying a number of nearby targets, you can perform a cone search, which will return all available targets within a cone of a specfied radius on the sky. The radius can be either a `float` or an [astropy.units.Quantity](https://docs.astropy.org/en/stable/units/) object. If a float is given, it will be assumed to be in units of arcseconds.  If `None` then we default to 0.0001 arcsec.

In [ ]:
search_result = lk.search_targetpixelfile('Trappist-1', radius=180., campaign=12, exptime=1800)
print(search_result)

The `distance` column describes the distance on the sky in arcseconds from the target or coordinates passed into the search.

## About this Notebook

**Authors**: Nicholas Saunders (nksaun@hawaii.edu)

**Updated**: September 28, 2020

## Citing Lightkurve and Astropy

If you use `lightkurve` or its dependencies in your published research, please cite the authors. Click the buttons below to copy BibTeX entries to your clipboard.

In [ ]:
lk.show_citation_instructions()

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>
